# I. Architecture des dossiers
## Cellule 1 (obligatoire)

In [1]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
warnings.simplefilter("ignore", FutureWarning, )

In [8]:
import os
chemin = r"G:\Esp-mission-Patrimoine\documentLibrary\Habitat_Amenagement\OBSERVATOIRE\03_AUTRES\ALTERNANT\ENZO\Avancement\fonctions\last_version"
os.chdir(chemin)
print(os.getcwd())

import sys
sys.path.append(r'G:\Esp-mission-Patrimoine\documentLibrary\Habitat_Amenagement\OBSERVATOIRE\03_AUTRES\ALTERNANT\ENZO\Avancement\fonctions\last_version')

import script_versement3
from script_versement3 import Fonctions3

import importlib
importlib.reload(script_versement3)
from script_versement3 import Fonctions3

g:\Esp-mission-Patrimoine\documentLibrary\Habitat_Amenagement\OBSERVATOIRE\03_AUTRES\ALTERNANT\ENZO\Avancement\fonctions\last_version


## Documentation du script

In [ ]:
help(Fonctions3)

## Cellule choupi (facultative)

In [ ]:
fonctions3.mignon(1,1,2)

## Cellule 2 (obligatoire)
### Initiation et choix de l'observatoire

In [9]:
fonctions = Fonctions3()

Demande connexion!
Liste des observatoires disponibles :
1 : l'ODH dans bdsociohab
2 : la MDDP
3 : l'ODH
4 : l'Observatoire de test
Connexion réussie!
Vous avez choisi l'observatoire l'Observatoire de test.
Vous vous apprêtez actuellement à faire votre versement dans la base de données obs et le schéma test.


## Cellule 3 (obligatoire)
### Renseignement du nom du fichier Excel à importer et de l'année de récolte des données

In [4]:
fonctions.nom_excel = "tuto.xlsx" # à modifier

fonctions.annee = 2025 # à modifier

if os.path.isfile(fonctions.nom_excel):
    print('Le fichier existe.')
else:
    print('Le fichier n\'existe pas.')

Le fichier existe.


# II. Utilisation du notebook

## II.1. Préparation des fichiers (facultatif)

### Cellule 4 (facultative)
### Fonction listant le nom des feuilles du fichier Excel (futur nom des variables dans pg)

In [8]:
fonctions.liste_feuille(fonctions.nom_excel) # Rien à changer

Variable(s) pour le jeu de données:
tr_age
tr_age_prop


### Cellule 5 (facultative)

### Fonction listant toutes les tables présentes dans le schéma de l'observatoire sélectionné
### + dico

In [16]:
liste = fonctions.liste_table(afficher='vrai') # Rien à changer

Tables existantes dans le schéma "test" sur PgAdmin:
_test_dict
_test_mod
_test_var
demog_menage_insee
dvf_volume_cerema
nombre_transac_dv3f


In [10]:
# (vous pouvez copier/coller les résultats de la cellule du dessus)
# nom complet de la table
fonctions.dict() # à modifier

Liste des dictionnaires disponibles :
 
1 : pour faire une recherche sur les modalités et les variables d'une table en particulier. 

2 : pour faire une recherche sur la table des modalités. 

3 : pour faire une recherche sur la table des variables. 

4 : pour faire une recherche sur le dictionnaire des tables. 

Tables existantes dans le schéma "test" sur PgAdmin:
Liste des tables disponibles :
_test_dict          | 0
_test_mod           | 1
_test_var           | 1
demog_menage_insee  | 2
dvf_volume_cerema   | 3
nombre_transac_dv3f | 5
ppu_sne_gipsne      | 8
theme_base_source   | 13


g:\Esp-mission-Patrimoine\documentLibrary\Habitat_Amenagement\OBSERVATOIRE\03_AUTRES\ALTERNANT\ENZO\Avancement\fonctions\last_version\script_versement3.py:1190: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_table = pd.read_sql_query(requete_sql, self.con)


Vous avez choisi: Non


,id_var,nom_var,joli_nom_var,lib_long_var,id_mod,nom_mod,joli_nom_mod,lib_long_mod
0,1,com_2014,com_2014,,1,men,MEN,
1,1,com_2014,com_2014,,2,menpseul,MENPSEUL,
2,1,com_2014,com_2014,,3,menhseul,MENHSEUL,
3,1,com_2014,com_2014,,4,menfseul,MENFSEUL,
4,1,com_2014,com_2014,,5,mensfam,MENSFAM,
5,1,com_2014,com_2014,,6,menfam,MENFAM,
6,1,com_2014,com_2014,,7,mencoupsenf,MENCOUPSENF,
7,1,com_2014,com_2014,,8,mencoupaenf,MENCOUPAENF,
8,1,com_2014,com_2014,,9,menfammono,MENFAMMONO,
9,1,com_2014,com_2014,,10,pmen,PMEN,


## Cellule 6 (obligatoire)
### Renseignement du nom de la source et de la base (futur nom de la base dans pg : source_base)
##### Veuillez prêter une attention particulière à l'orthographe si la table existe déjà et que vous voulez la remplir. Pour cela, vous pouvez vous servir de la fonction listant toutes les tables présentes dans le schéma de l'observatoire

In [5]:
fonctions.theme = 'theme' # à modifier

fonctions.base = 'base' # à modifier

fonctions.source = 'source' # à modifier

print(f"Le nom de la table sera: \033[1m{fonctions.nom_table}\033[0m.")

Le nom de la table sera: theme_base_source.


## Cellule 7 (obligatoire)
#### Modifications automatisées sur le jeu de données à verser

In [ ]:
fonctions.modiFichier(fonctions.nom_excel) # Rien à changer

## Cellule 8 (obligatoire)
### Fonction de suppression des fichiers temporaires

In [6]:
fonctions.suppression_fichiers(vars=False) # à modifier

Le dossier fichiers_brutes_csv a été supprimé.
Le dossier fichiers_traites a été supprimé.


## Cellule 9 (obligatoire)
### Première fonction de versement

#### Cette fonction à préparer les données à importer plus tard. Il créer, dans un csv, les tables valeur, modalité et variable dans la bonne forme

In [7]:
fonctions.traitement() # Rien à changer

Liste des observatoires disponibles :
1 : Commune
2 : EPCI
3 : IRIS
4 : Canton
5 : Logement
6 : Parcelle
7 : Section cadastrale
8 : Département
9 : Autre
Vous avez choisi l'échelle Commune.
Vous avez répondu : N
Fichiers temporaires créés.
Importation des feuilles Excel en cours...
Importation des feuilles Excel terminée.
Traitement du dictionnaire terminé.
variable recherchée : var1
nouvelle
la variable var1 a été insérée
variable recherchée : var34
nouvelle
la variable var34 a été insérée
Traitement des variables terminé.


100%|██████████| 2/2 [00:00<00:00, 285.64it/s]

__________________________________________________
nom de la feuille en cours de traitement: var1
Modalité recherchée : men
trouvée
Modalité recherchée : col2
nouvelle
__________________________________________________
nom de la feuille en cours de traitement: var34
Modalité recherchée : col3
nouvelle
Modalité recherchée : col34
nouvelle


Traitement des valeurs et des modalités terminé.
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
Opération terminée: les fichiers temporaires ont été créé dans le dossier vars_2024253144640 
Vous pouvez désormais verser les données grâce à la cellule 10 ou les vérifier dans le dossier vars et fichiers_traites.
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+


## Cellule 10 (obligatoire)
### Deuxième fonction de versement
#### Cette fonction importe dans pgAdmin les tables valeur, modalité et variable créées par la première fonction

In [9]:
fonctions.nom_dossier_vars

'vars_2024253144640'

In [11]:
fonctions.versement() # Rien à changer

Début du versement...
____________________________________________________________________________________________________
Préparation de la table dictionnaire...
Le temps pour préparer le versement des données de la table dictionnaire a été de 0.004 secondes
Les données sont prêtes à être insérées dans la table dictionnaire.
____________________________________________________________________________________________________
Préparation de la table variable...
Traceback (most recent call last):
  File "g:\Esp-mission-Patrimoine\documentLibrary\Habitat_Amenagement\OBSERVATOIRE\03_AUTRES\ALTERNANT\ENZO\Avancement\fonctions\last_version\script_versement3.py", line 711, in versement
    self.insertion_donnees(table)
  File "g:\Esp-mission-Patrimoine\documentLibrary\Habitat_Amenagement\OBSERVATOIRE\03_AUTRES\ALTERNANT\ENZO\Avancement\fonctions\last_version\script_versement3.py", line 855, in insertion_donnees
    self.cur.execute(f"INSERT INTO {self.livre[table]['nom_table']} VALUES " + arg

In [90]:
fonctions.suppression_table('theme_base_source', id_versement_cible=6, garder_table=False)

Vous avez choisi: Oui
self.con.autocommit: False
Requête:
DELETE FROM test.theme_base_source WHERE id_versement = 6
Requête:
DELETE FROM _test_dict WHERE id_versement = 6
Requête:
DELETE FROM _test_var WHERE id_var in (8)
Requête:
DELETE FROM _test_mod WHERE id_mod in (84)
Opération réussie.
